In [1]:
import keras.callbacks
import tensorflow as tf
from keras.layers import Dense, Input, Dropout, MaxPooling1D, Conv1D, GlobalMaxPool1D
from keras.layers import LSTM, Lambda, concatenate, TimeDistributed, Bidirectional

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.layers import Input, Embedding, Activation, Flatten, Dense
from keras.layers import Conv1D, MaxPooling1D, Dropout
from keras.models import Model
from keras.layers import LeakyReLU

train_data_source = '/content/drive/My Drive/nlp_assignment_3/train.csv'
test_data_source = '/content/drive/My Drive/nlp_assignment_3/test.csv'

train_df = pd.read_csv(train_data_source, delimiter="\t")
test_df = pd.read_csv(test_data_source, delimiter="\t")

In [4]:
train_df.head()

,meta,uid,sentiment
0,meta,3,negative
1,@,O,NaN
2,AdilNisarButt,Hin,NaN
3,pakistan,Hin,NaN
4,ka,Hin,NaN


In [5]:
test_df.head()

,meta,uid,sentiment
0,meta,8,neutral
1,RT,Eng,NaN
2,@,O,NaN
3,UAAPconfessions,Eng,NaN
4,Love,Eng,NaN


In [6]:
string = ""
a = train_df.loc[:,['meta','sentiment']]
temp = ""
train_arr = []
train_arr_senti = []

for i in range(a.shape[0]):
  if(a.iloc[i,0]=="meta"):
    #print(a.iloc[i,0])
    train_arr.append(temp)
    train_arr_senti.append(str(a.iloc[i,1]))
    temp=""
  else:
    temp+=str(a.iloc[i,0])+" "

train_arr.append(temp)
train_arr=train_arr[1:]
print(train_arr[0],train_arr_senti[0])

@ AdilNisarButt pakistan ka ghra tauq he Pakistan Israel ko tasleem nahein kerta Isko Palestine kehta he - OCCUPIED PALESTINE  negative


In [7]:
string = ""
a = test_df.loc[:,['meta','sentiment']]
temp = ""
test_arr = []
test_arr_senti = []

for i in range(a.shape[0]):
  if(a.iloc[i,0]=="meta"):
    #print(a.iloc[i,0])
    test_arr.append(temp)
    test_arr_senti.append(str(a.iloc[i,1]))
    temp=""
  else:
    temp+=str(a.iloc[i,0])+" "

test_arr.append(temp)
test_arr=test_arr[1:]
print(test_arr[0],test_arr_senti[0])

RT @ UAAPconfessions Love looks good on Maddie !!! Ako lang ba yung sobrang masaya kasi may zolo sya ? Before with the past Z medyo lowkey s …  neutral


In [8]:
# convert string to lower case
train_texts = train_arr[:]
train_texts = [s.lower() for s in train_texts]

test_texts = test_arr[:]
test_texts = [s.lower() for s in test_texts]

# =======================Convert string to index================
# Tokenizer
tk = Tokenizer(num_words=None, char_level=True, oov_token='UNK')
tk.fit_on_texts(train_texts)

# Convert string to index
train_sequences = tk.texts_to_sequences(train_texts)
test_texts = tk.texts_to_sequences(test_texts)

# Padding
train_data = pad_sequences(train_sequences, maxlen=160, padding='post')
test_data = pad_sequences(test_texts, maxlen=160, padding='post')

# Convert to numpy array
train_data = np.array(train_data, dtype='float32')
test_data = np.array(test_data, dtype='float32')

# =======================Get classes================
train_classes_1 = train_arr_senti[:]
train_class_list = []

for i in train_classes_1:
  if(i == "positive"):
    train_class_list.append(0)
  elif(i == "negative"):
    train_class_list.append(2)
  else:
    train_class_list.append(1)

train_class_list = np.array(train_class_list,dtype='float32')

test_classes_1 = test_arr_senti[:]
test_class_list = []

for i in test_classes_1:
  if(i == "positive"):
    test_class_list.append(0)
  elif(i == "negative"):
    test_class_list.append(2)
  else:
    test_class_list.append(1)

test_class_list = np.array(test_class_list,dtype='float32')

from keras.utils import to_categorical

train_classes = to_categorical(train_class_list)
test_classes = to_categorical(test_class_list)
print(train_classes[0])
print(test_classes[0])

[0. 0. 1.]
[0. 1. 0.]


In [0]:
from keras.layers import GRU,LSTM

In [28]:
# =====================Char CNN=======================
# parameter
input_size = 160
vocab_size = len(tk.word_index)
print(tk.word_index)
embedding_size = 900
conv_layers = [[256, 3, 3],
               [256, 3, 3],
               [256, 3, -1],
               [256, 3, -1],
               [256, 3, -1],
               [256, 3, 3]]

fully_connected_layers = [1024, 1024]
num_of_classes = 3
dropout_p = 0.5
optimizer = 'adadelta'
loss = 'categorical_crossentropy'

# Embedding weights
embedding_weights = [] 
embedding_weights.append(np.zeros(vocab_size))  

for char, i in tk.word_index.items():
    onehot = np.zeros(vocab_size)
    onehot[i - 1] = 1
    embedding_weights.append(onehot)

embedding_weights = np.array(embedding_weights)


# Embedding layer Initialization
embedding_layer = Embedding(vocab_size + 1,embedding_size, weights=[embedding_weights])

# Model Construction

# Input
inputs = Input(shape=(input_size,), name='input', dtype='int64') 

# Embedding
x = embedding_layer(inputs)

# Conv
for filter_num, filter_size, pooling_size in conv_layers:
    x = Conv1D(filter_num, filter_size)(x)
    x = LeakyReLU(alpha=0.05)(x)
    if pooling_size != -1:
        x = MaxPooling1D(pool_size=pooling_size)(x)  

x = LSTM(300, dropout = 0.4, recurrent_dropout = 0.4)(x)

# x = Flatten()(x)

# Fully connected layers
for dense_size in fully_connected_layers:
    x = Dense(dense_size, activation='linear')(x)
    x = LeakyReLU(alpha=0.05)(x)
    x = Dropout(dropout_p)(x)

# Output Layer
predictions = Dense(num_of_classes, activation='softmax')(x)

# Build model
model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])  # Adam, categorical_crossentropy
model.summary()

# Shuffle
indices = np.arange(train_data.shape[0])
np.random.shuffle(indices)

x_train = train_data[indices]
y_train = train_classes[indices]

x_test = test_data
y_test = test_classes

{'UNK': 1, ' ': 2, 'a': 3, 'i': 4, 'e': 5, 'h': 6, 'n': 7, 't': 8, 'o': 9, 'r': 10, 's': 11, 'k': 12, 'm': 13, 'l': 14, '\t': 15, '\n': 16, 'd': 17, 'u': 18, 'g': 19, 'p': 20, 'b': 21, 'y': 22, 'c': 23, '/': 24, '@': 25, '.': 26, 'j': 27, 'w': 28, 'f': 29, 'v': 30, '…': 31, 'z': 32, '1': 33, '_': 34, '0': 35, '4': 36, '2': 37, 'q': 38, '3': 39, 'x': 40, '9': 41, '7': 42, '5': 43, '8': 44, '6': 45, '#': 46, '!': 47, '😂': 48, '?': 49, "'": 50, '-': 51, '’': 52, '️': 53, '🙏': 54, '❤': 55, '🤣': 56, '😍': 57, '&': 58, ')': 59, '(': 60, '😭': 61, '*': 62, '😘': 63, '🇮': 64, '🇳': 65, '😊': 66, '🌹': 67, '💜': 68, '🎂': 69, '“': 70, '💕': 71, 'ा': 72, '😁': 73, '👏': 74, '🎉': 75, '💖': 76, '👍': 77, '|': 78, '👌': 79, '”': 80, '✌': 81, '😜': 82, '%': 83, 'र': 84, '😆': 85, '💐': 86, '🏻': 87, '+': 88, '♥': 89, '🤗': 90, '्': 91, '😎': 92, '😡': 93, '🙄': 94, '🔥': 95, '~': 96, '🤔': 97, '\u200d': 98, '😉': 99, '💙': 100, '😠': 101, '😅': 102, '🙌': 103, '\U0001f92a': 104, '😀': 105, 'न': 106, 'क': 107, 'म': 108, ';': 109,

In [0]:
###### To Calculate Precision, Recall, F1-Score over All Classes
# Class1 --- Positive
# Class2 --- Neutral
# Class3 --- Negative
 
import numpy as np
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score

class Metrics(Callback):
  def __init__(self, logs={}):
    self.val_f1s = []
    self.val_recalls = []
    self.val_precisions = []
 
  def on_epoch_end(self, epoch, logs={}):
    val_predict = (np.asarray(self.model.predict(x_test))).round()
    val_targ = y_test
    _val_f1 = f1_score(val_targ, val_predict,average=None)
    _val_recall = recall_score(val_targ, val_predict,average=None)
    _val_precision = precision_score(val_targ, val_predict,average=None)
    self.val_f1s.append(_val_f1)
    self.val_recalls.append(_val_recall)
    self.val_precisions.append(_val_precision)
    print(" — val_f1_Class1: %f — val_f1_Class2: %f — val_f1_Class3: %f — val_precision_Class1: %f — val_precision_Class2: %f — val_precision_Class3: %f — val_recall_Class1 %f — val_recall_Class2 %f — val_recall_Class3 %f" %(_val_f1[0], _val_f1[1], _val_f1[2], _val_precision[0], _val_precision[1], _val_precision[2], _val_recall[0], _val_recall[1], _val_recall[2]))
    return
 
metrics = Metrics()

In [31]:
model.fit(x_train, y_train,validation_data=(x_test, y_test),batch_size=128,epochs=30,verbose=1,callbacks=[metrics])

Train on 13524 samples, validate on 1869 samples
Epoch 1/30
13524/13524 [==============================] - 3s 254us/step - loss: 0.3189 - acc: 0.8873 - val_loss: 1.6987 - val_acc: 0.5104
 — val_f1_Class1: 0.558502 — val_f1_Class2: 0.464462 — val_f1_Class3: 0.505451 — val_precision_Class1: 0.511429 — val_precision_Class2: 0.494753 — val_precision_Class3: 0.535714 — val_recall_Class1 0.615120 — val_recall_Class2 0.437666 — val_recall_Class3 0.478424
Epoch 2/30
13524/13524 [==============================] - 3s 253us/step - loss: 0.2956 - acc: 0.8985 - val_loss: 1.7727 - val_acc: 0.4949
 — val_f1_Class1: 0.290713 — val_f1_Class2: 0.580247 — val_f1_Class3: 0.457859 — val_precision_Class1: 0.670807 — val_precision_Class2: 0.451923 — val_precision_Class3: 0.582609 — val_recall_Class1 0.185567 — val_recall_Class2 0.810345 — val_recall_Class3 0.377111
Epoch 3/30
13524/13524 [==============================] - 3s 253us/step - loss: 0.2633 - acc: 0.9112 - val_loss: 1.8776 - val_acc: 0.5062
 — val_